# Babble Tutorial

### You will work with Wikipedia plot descriptions of films that are either comedy or drama.


In [2]:
from data.preparer import load_film_dataset

## The Data

These plot descriptions are from [Kaggle](https://www.kaggle.com/jrobischon/wikipedia-movie-plots).
You will be labeling films as either "comedy" or "drama" based on their plot descriptions.

In [3]:
df_train, df_dev, df_valid, df_test = load_film_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_test)))

8343 training examples
500 development examples
500 validation examples
1000 test examples


In [4]:
#define labels
ABSTAIN = 0
DRAMA = 1
COMEDY = 2

Transform the data into a format compatible with Babble Labble:

In [5]:
from babble.Candidate import Candidate # this is a helper class to transform our data into a format Babble can parse

dfs = [df_train, df_dev, df_test]

for df in dfs:
    df["id"] = range(len(df))

Cs = [df.apply(lambda x: Candidate(x), axis=1) for df in dfs]

# babble labble uses 1 and 2 for labels, while our data uses 0 and 1
# add 1 to convert
Ys = [df.label.values + 1 for df in dfs]

In [6]:
from babble import BabbleStream

aliases = {}
babbler = BabbleStream(Cs, Ys, balanced=True, shuffled=True, seed=456, aliases=aliases)

Grammar construction complete.


## Create Explanations

Creating explanations generally happens in five steps:
1. View candidates
2. Write explanations
3. Get feedback
4. Update explanations 
5. Apply label aggregator

Steps 3-5 are optional; explanations may be submitted without any feedback on their quality. However, in our experience, observing how well explanations are being parsed and what their accuracy/coverage on a dev set are (if available) can quickly lead to simple improvements that yield significantly more useful labeling functions. Once a few labeling functions have been collected, you can use the label aggregator to identify candidates that are being mislabeled and write additional explanations targeting those failure modes.

### Collection

Use `babbler` to show candidates

In [8]:
candidate = babbler.next()
print(candidate)

{'text': "Lorna Bellstratten (Walters), a waitress with dreams of being in show business, is duped by her drug-dealer boyfriend Michael Vega (Guastaferro) into delivering a bomb to an undercover cop. Though Lorna survives the explosion (intended to kill her and the cop), she finds herself—as the only material witness to the crime she unwittingly abetted—wanted by both the cops and the mob (Vega's employers). Distraught, Lorna flees to Cabo San Lucas, Mexico and takes out a contract on her own life (suicide-by-hitman.) Meanwhile, Vega (posing as Lorna's father) hires Los Angeles bail bondsman, Eddie Moscone (Hedaya) to send in a bounty hunter to bring her back to LA alive. Eddie offers the job to bounty hunter Jack Walsh (McDonald) for $10,000. He doesn't want to take the job because Eddie keeps on stiffing him his money. Eddie threatens to give the job to rival bounty hunter Marvin Dorfler, who does not make an appearance. When Walsh finds her in Cabo San Lucas, Lorna thinks he's her h

In [10]:
explanations = []

In [11]:
from babble import Explanation
explanation = Explanation(
    name='bounty_hunter', # name of this rule, for your reference
    label=COMEDY, # label to assign
    condition='The phrase "bounty hunter" is in the text', # natural language description of why you label the candidate this way
    candidate=candidate.mention_id # optional argument, the candidate should be an example labeled by this rule
)
explanations.append(explanation)

Babble will parse your explanations into functions, then filter out functions that are duplicates, incorrectly label their given candidate, or assign the same label to all examples.

In [12]:
parses, filtered = babbler.apply(explanations)

Building list of target candidate ids...
Collected 1 unique target candidate ids from 1 explanations.
Gathering desired candidates...
Found 1/1 desired candidates
Linking explanations to candidates...
Linked 1/1 explanations
1 explanation(s) out of 1 were parseable.
1 parse(s) generated from 1 explanation(s).
1 parse(s) remain (0 parse(s) removed by DuplicateSemanticsFilter).
Note: 1 LFs did not have candidates and therefore could not be filtered.
1 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%

1 parse(s) remain (0 parse(s) removed by UniformSignatureFilter: (0 None, 0 All)).
1 parse(s) remain (0 parse(s) removed by DuplicateSignatureFilter).
1 parse(s) remain (0 parse(s) removed by LowestCoverageFilter).


### Analysis
See how your explanations were parsed and filtered

In [13]:
babbler.analyze(parses)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
bounty_hunter_0,0,2.0,0.002,0.0,0.0,1,0,1.0


In [14]:
babbler.filtered_analysis(filtered)

No filtered parses to analyze.


In [15]:
babbler.commit()

Added 1 parse(s) from 1 explanations to set. (Total # parses = 1)

Applying labeling functions to split 1
[========================================] 100%

Added 1 labels to split 1: L.nnz = 1, L.shape = (500, 1).
Applying labeling functions to split 2
[========================================] 100%

Added 1 labels to split 2: L.nnz = 1, L.shape = (1000, 1).


### Evaluation
Get feedback on the performance of your explanations

In [16]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], Ys[1], lf_names=lf_names)

Retrieved label matrix for split 0: L.nnz = 6, L.shape = (8343, 1)
Retrieved label matrix for split 1: L.nnz = 1, L.shape = (500, 1)
Retrieved label matrix for split 2: L.nnz = 1, L.shape = (1000, 1)


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
bounty_hunter_0,0,2,0.002,0.0,0.0,1,0,1.0


In [17]:
from metal import LabelModel
from metal.tuners import RandomSearchTuner

search_space = {
    'n_epochs': [50, 100, 500],
    'lr': {'range': [0.01, 0.001], 'scale': 'log'},
    'show_plots': False,
}

tuner = RandomSearchTuner(LabelModel, seed=123)

label_aggregator = tuner.search(
    search_space, 
    train_args=[Ls[0]], 
    X_dev=Ls[1], Y_dev=Ys[1], 
    max_search=20, verbose=False, metric='f1')

[SUMMARY]
Best model: [1]
Best config: {'n_epochs': 500, 'show_plots': False, 'lr': 0.0012223249524949424, 'seed': 123}
Best score: 0.7221510883482715


If you'd like to save the explanations you've generated, you can use the `ExplanationIO` object to write to or read them from file.

In [18]:
from babble.utils import ExplanationIO

FILE = "my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)

Wrote 1 explanations to my_explanations.tsv
Read 1 explanations from my_explanations.tsv
